#### Defined by User

In [ ]:
%store -r
import os
from urllib.parse import urlparse

## Project ID, make sure IDE is created under project
## Or else provide it manually in the project_id variable.
project_id = os.environ.get("DKUBE_PROJECT_ID")

## set this to false if you are running the notebook other than
## the setup where deployment is running or monitoring on other cluster
RUNNING_IN_SAME = True

######## Dkube information of serving cluster ########

## if RUNNING_IN_SAME is False then provide
## Serving Dkube URL with expernal IP, eg: https://{IP}:32222/
## in else part
# if your SERVING_DKUBE_URL dosn't have IP the provide the value for SERVING_DKUBE_IP mannually.
if RUNNING_IN_SAME:
    MINIO_ENDPOINT = os.getenv("MLFLOW_S3_ENDPOINT_URL")
    SERVING_DKUBE_URL = os.getenv("DKUBE_URL")
else:
    SERVING_DKUBE_URL = ""
    parsed_url = urlparse(SERVING_DKUBE_URL)
    SERVING_DKUBE_IP = parsed_url.hostname
    MINIO_ENDPOINT = f"http://{SERVING_DKUBE_IP}:32221"


if (not SERVING_DKUBE_URL) or (not MINIO_ENDPOINT):
    raise ValueError("Either SERVING_DKUBE_URL or MINIO_ENDPOINT is empty")

SERVING_DKUBE_USERNAME = os.getenv("DKUBE_USER_LOGIN_NAME","")

SERVING_DKUBE_TOKEN = os.getenv("DKUBE_USER_ACCESS_TOKEN","") 

######## Dkube information of monitoring cluster ########

## The following values are not required if 
## monitoring and serving is running in same Dkube Cluster
MONITORING_DKUBE_URL = ""

MONITORING_DKUBE_USERNAME = ""

MONITORING_DKUBE_TOKEN = ""

## name of serving cluster
SERVING_DKUBE_CLUSTER_NAME = ""

## Define the model monitor name here that you will be creating 
## it should be same as the deployment name which is running on DKube
## by default it will get picked from the pipeline run
## if not present provide manually. 
if "pl_config" in dir():
    MONITOR_NAME = pl_config.get("DEPLOYMENT_NAME")
else:
    MONITOR_NAME = ""

# Required in train.ipynb to retrain the model
LIVE_DATASET =  MONITOR_NAME+'-s3'

## Inference URL
INFERENCE_URL = None

# dataset to be used as training data
DKUBE_BASE_DATASET = "insurance-data"

# Model name to be created or used for example, it will create the model
# if not existing otherwise it will ignore creation.
MODEL_NAME = MONITOR_NAME

# the frequency with which monitoring will run, value will be considered in minutes
RUN_FREQUENCY = 5

if project_id:
    tags = [f"project:{project_id}"]
else:
    tags = []

if not MODEL_NAME:
    raise Exception("Model name is empty")

if not(SERVING_DKUBE_TOKEN and SERVING_DKUBE_USERNAME and SERVING_DKUBE_URL):
    raise Exception("Please fill the Serving Dkube details first (SERVING_DKUBE_TOKEN, SERVING_DKUBE_URL, SERVING_DKUBE_USERNAME)")
    
if (MONITORING_DKUBE_URL
   and
   not(MONITORING_DKUBE_USERNAME and MONITORING_DKUBE_TOKEN and SERVING_DKUBE_CLUSTER_NAME)):
    raise Exception("Please fill the Monitoring Dkube details first (MONITORING_DKUBE_USERNAME, MONITORING_DKUBE_TOKEN, MONITORING_DKUBE_URL, SERVING_DKUBE_CLUSTER_NAME)")

#### Dkube Resources

In [ ]:
import time,json
from dkube.sdk import *

In [ ]:
serving_api = DkubeApi(URL=SERVING_DKUBE_URL,token=SERVING_DKUBE_TOKEN)
if SERVING_DKUBE_USERNAME == serving_api.validate_token()['username']:
    pass
else:
    print("Invalid User, please check your username, first")

In [ ]:
if not MONITORING_DKUBE_URL:
    data = serving_api.get_modelmonitor_id(MONITOR_NAME)
    if data.data:
        MONITOR_ID = data.data.get(MONITOR_NAME)
        if MONITOR_ID:
            raise ValueError(f"{MONITOR_NAME} monitor already existing please use a different name")

In [ ]:
response =  serving_api.get_cloudevents_logstore_creds()
MINIO_KEY = response["access_key_id"]
MINIO_SECRET_KEY = response["access_key"]
MINIO_BUCKET = response["bucket"]

In [ ]:
INPUT_TRAIN_TYPE = "training"
cld_config = {"MONITOR_NAME":MONITOR_NAME,
              "INPUT_TRAIN_TYPE":INPUT_TRAIN_TYPE, "SERVING_DKUBE_USERNAME":SERVING_DKUBE_USERNAME,
              "SERVING_DKUBE_URL":SERVING_DKUBE_URL, "SERVING_DKUBE_TOKEN":SERVING_DKUBE_TOKEN, 
              "MINIO_KEY":MINIO_KEY,"MINIO_SECRET_KEY":MINIO_SECRET_KEY, "MINIO_ENDPOINT":MINIO_ENDPOINT,
              "MINIO_BUCKET": MINIO_BUCKET,
              "DKUBE_BASE_DATASET":DKUBE_BASE_DATASET, "MODEL_NAME":MODEL_NAME,
              "RUN_FREQUENCY":RUN_FREQUENCY,
              "LIVE_DATASET":LIVE_DATASET,
              "MONITORING_DKUBE_USERNAME":MONITORING_DKUBE_USERNAME, "MONITORING_DKUBE_TOKEN":MONITORING_DKUBE_TOKEN,
              "MONITORING_DKUBE_URL":MONITORING_DKUBE_URL,
              "SERVING_DKUBE_CLUSTER_NAME":SERVING_DKUBE_CLUSTER_NAME}

#### Dataset

In [ ]:
try:
    dataset = DkubeDataset(SERVING_DKUBE_USERNAME, name=DKUBE_BASE_DATASET, tags=tags)
    dataset.update_git_details(url="https://dkube-examples-data.s3.us-west-2.amazonaws.com/monitoring-insurance/training-data/insurance.csv")
    dataset.update_dataset_source(source="pub_url")
    serving_api.create_dataset(dataset)
except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")

## Checking for seperate monitoring cluster and adding datasets accordingly.

In [ ]:
if MONITORING_DKUBE_URL:
    monitoring_api = DkubeApi(URL=MONITORING_DKUBE_URL,token=MONITORING_DKUBE_TOKEN)
    data = monitoring_api.get_modelmonitor_id(MONITOR_NAME)
    if data.data:
        MONITOR_ID = data.data.get(MONITOR_NAME)
        if MONITOR_ID:
            raise ValueError(f"{MONITOR_NAME} monitor already existing please use a different name")
    try:
        dataset = DkubeDataset(MONITORING_DKUBE_USERNAME, name=DKUBE_BASE_DATASET)
        dataset.update_git_details(url="https://dkube-examples-data.s3.us-west-2.amazonaws.com/monitoring-insurance/training-data/insurance.csv")
        dataset.update_dataset_source(source="pub_url")
        monitoring_api.create_dataset(dataset)
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")

### Model Monitor Datasets

##### Predict and labelled Dataset

In [ ]:
if MONITORING_DKUBE_URL:
    api = monitoring_api
    user = MONITORING_DKUBE_USERNAME
else:
    api = serving_api
    user = SERVING_DKUBE_USERNAME

In [ ]:
try:
    dataset = DkubeDataset(user, name=LIVE_DATASET,remote=True,tags=tags)
    dataset.update_dataset_source('s3')
    dataset.update_s3_details(
        endpoint = MINIO_ENDPOINT,
        bucket=MINIO_BUCKET,
        prefix='',
        key=MINIO_KEY,
        secret=MINIO_SECRET_KEY)
    api.create_dataset(dataset)

except Exception as e:
    if e.reason:
        if e.reason.lower() != "conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
    else:
        raise e

## Waiting for the deployment to come up and enabling logs

In [ ]:
del api, serving_api

# Delete monitoring_api variable if it was created earlier by checking for external monitoring url being set
if MONITORING_DKUBE_URL: del monitoring_api

serving_api = DkubeApi(URL=SERVING_DKUBE_URL,token=SERVING_DKUBE_TOKEN)
def wait_for_deployment_running(deployment_id):
    status = None
    inference_url, inference = None, None
    while True:
        data = serving_api.get_deployment(deployment_id)
        status = data.data.inferenceservice_deployment.parameters.generated.status.state
        inference = data.data.inferenceservice_deployment.parameters.inference
        inference_url = data.data.inferenceservice_deployment.parameters.generated.details.serving.servingurl
        if status == "RUNNING":
            break
        print("waiting for deployment to be running")
        time.sleep(serving_api.wait_interval)
    return inference, inference_url

while True:
    SERVING_DEPLOYMENT_ID = serving_api.get_deployment_id(name=MONITOR_NAME)
    if SERVING_DEPLOYMENT_ID:
        break
    print("waiting for deployment to come up")
    time.sleep(serving_api.wait_interval)
inference, INFERENCE_URL = wait_for_deployment_running(SERVING_DEPLOYMENT_ID)
if not inference.enable_logs:
    print("Enabling logs")
    serving = DkubeServing(user=SERVING_DKUBE_USERNAME, name=MONITOR_NAME)
    serving.update_enable_logs(enable_logs=True)
    serving_api.update_inference(serving)
print("Inference is up at URL: ", INFERENCE_URL)
cld_config['INFERENCE_URL'] = INFERENCE_URL
cld_config['SERVING_DEPLOYMENT_ID'] = SERVING_DEPLOYMENT_ID
%store cld_config

#### Cleanup

In [ ]:
## Set CLEANUP = True, after your experiment is complete.
CLEANUP = False
if CLEANUP:
    serving_api.delete_dataset(SERVING_DKUBE_USERNAME,DKUBE_BASE_DATASET,force=True)
    serving_api.delete_dataset(SERVING_DKUBE_USERNAME,LIVE_DATASET,force=True)
    %store -d cld_config